# Exploration_SSAC 08 Movielens 영화 추천

**Keyword** : 추천시스템 알고리즘, 협업 필터링/콘텐츠 기반 필터링, Matrix Factorization(MF), CSR(Compressed Sparse Row) Matrix

##### Project Process  

1) 데이터 준비와 전처리  
2) 데이터 분석  
3) CSR Matrix 생성  
4) AlternatingLeastSquares Model 생성  
5) 모델 예측  
6) 모델 활용

##### Project Preview  
* Dataset : MovieLens 1M Dataset  
* rating.dat 파일에는 이미 인덱싱이 완료된 사용자-영화-평점 데이터 정제 완료
* Explicit 데이터지만 Implicit 데이터로 간주하고 테스트  
* Explicit (별점 데이터)를 '시청횟수'로 해석하여 진행  
* 데이터 value 가 3점 미만 (별점)이면, 사용자가 선호하지 않은 것이라 가정 (제외 데이터)

##### 필요한 모듈 import

In [62]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 1) 데이터 준비와 전처리

* .dat 파일 형식이 무엇인지 몰랐는데, 직접 파일을 열어보니 Dataframe 형태가 아닌 텍스트와 부호로 이루어진 파일이었다.   
* DAT 파일에 포함된 정보는 일반적으로 일반 텍스트 또는 이진이라고 한다.  
* 따라서, 데이터프레임 형태로 만들어 주기 위해, 직접 컬럼명을 지정해 준다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp'] # 컬럼명 지정
# 본래 .dat 파일이기 때문에, sep, engine의 파라미터 활용하여 변환
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings) # 1000209
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratings.shape

(1000209, 4)

In [4]:
# 3점 이상만 남깁니다. 위에서 가정한대로 적용
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings) # 836478

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# 모델 생성시, '별점'이 아닌 영화 '시청 횟수'로 가정하고 진행하기 위해 컬럼명 수정
ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터 로딩
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies.shape

(3883, 3)

In [9]:
# 모델 학습 후, 검증하는 단계에서 쉬운 검색을 위해 모두 소문자로 변경
movies['title'] = movies['title'].str.lower()
movies.head(5)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [129]:
movies[movies['movie_id'] == 3]['title']

2    grumpier old men (1995)
Name: title, dtype: object

## 2) 데이터 분석

* ratings에 있는 유니크한 영화 개수  
* ratings에 있는 유니크한 사용자 수   
* 가장 인기 있는 영화 30개 (인기수) -> 시청 횟수 평균이 가장 높은 영화 (?)

**1) ratings에 있는 유니크한 영화 개수 :: 3628개의 영화 기준**

In [10]:
ratings['movie_id'].nunique()

3628

**2) ratings에 있는 유니크한 사용자수 :: 6039명의 사용자 대상**

In [11]:
ratings['user_id'].nunique()

6039

In [12]:
ratings['user_id'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040])

**3) 가장 인기 있는 영화 30개 (인기수)**

In [13]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
pd.DataFrame(movie_count.sort_values(ascending=False)).head(30)

,user_id
movie_id,
2858,3211
260,2910
1196,2885
1210,2716
2028,2561
589,2509
593,2498
1198,2473
1270,2460


**4) 사용자별 몇 개의 영화를 봤는지**

In [14]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

* 전체 사용자는 6039명  
* 사용자의 시청한 영화 개수 평균 : 138편의 영화  
* 가장 많은 영화를 시청한 사용자의 영화 수 : 3628편 중 1968평 시청

**5) movie_id와 title를 한번에 볼 수 있도록 merge table 생성**

In [15]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [17]:
ratings.tail()

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [19]:
ratings.dtypes

user_id      int64
movie_id     int64
count        int64
timestamp    int64
dtype: object

In [16]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


서로 공통된 열이름 (movie_id)을 기준으로 inner(교집합) 조인을 하게 된다.

In [20]:
merge_ratings = pd.merge(ratings, movies)
merge_ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),Drama


In [21]:
movies.title.unique()

array(['toy story (1995)', 'jumanji (1995)', 'grumpier old men (1995)',
       ..., 'tigerland (2000)', 'two family house (2000)',
       'contender, the (2000)'], dtype=object)

In [22]:
# 모델 검증을 위한 사용자 정보 세팅을 하기 위해 최근에 나온 영화가 리스트에 있는지 확인하려 했지만
# 2000년 까지의 영화만 있는 것을 확인
movies[movies['title'].str.contains('1999')]

,movie_id,title,genre
2166,2235,one man's hero (1999),Drama|War
2367,2436,tea with mussolini (1999),Comedy
2376,2445,at first sight (1999),Drama
2377,2446,in dreams (1999),Thriller
2378,2447,varsity blues (1999),Comedy|Drama
...,...,...,...
3797,3867,all the rage (a.k.a. it's the rage) (1999),Drama
3824,3894,solas (1999),Drama
3832,3902,goya in bordeaux (goya en bodeos) (1999),Drama
3837,3907,"prince of central park, the (1999)",Drama


##### 모델 검증을 위한 사용자 초기 정보 세팅

{선호하는 영화 리스트}

3225 down to you (2000)    
3190 supernova (2000)    
1 toy story (1995)    
2 jumanji (1995)    
2445 at first sight (1999)

In [24]:
ratings.tail()

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [26]:
my_favorite = [3225 , 3190 ,1 ,2 ,2445]

# '6041'(== my user_id)이라는 user_id가 위 영화를 5회씩(count) 봤다고 가정
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'count':5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 '6041'이라는 데이터가 없다면
    ratings = ratings.append(my_movie)                       # 위에 임의로 만든 my_favorite 데이터를 추가

ratings.tail(10)

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,3225,5,NaN
1,6041,3190,5,NaN
2,6041,1,5,NaN
3,6041,2,5,NaN
4,6041,2445,5,NaN


In [28]:
ratings.dtypes

user_id        int64
movie_id       int64
count          int64
timestamp    float64
dtype: object

In [36]:
ratings.movie_id.max()

3952

In [29]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [45]:
print(user_to_idx[6041])    
print(movie_to_idx[3949])

6039
1318


In [52]:
print(user_unique.shape)

(6040,)


In [47]:
ratings.dtypes

user_id        int64
movie_id       int64
count          int64
timestamp    float64
dtype: object

In [56]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_artist_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_artist_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing Fail!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,2850,5,NaN
1,6039,2133,5,NaN
2,6039,40,5,NaN
3,6039,513,5,NaN


## 3) CSR Matrix 생성

In [58]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 4) AlternatingLeastSquares Model 생성

In [60]:
# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [85]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=70, dtype=np.float32)

In [86]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [87]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/70 [00:00<?, ?it/s]

## 5) 모델 예측

##### 모델이 벡터를 어떻게 생성하고 있는지 확인

In [88]:
mymine, at_first_sight = user_to_idx[6041], movie_to_idx[2445]
mymine_vector, at_first_sight_vector = als_model.user_factors[mymine], als_model.item_factors[at_first_sight]

print('슝=3')

슝=3


In [89]:
mymine_vector

array([-2.46210992e-01,  6.58463612e-02,  2.46189106e-02, -2.43816022e-02,
        2.24307507e-01,  9.78744216e-03,  9.88938361e-02, -3.69759463e-03,
       -2.80699909e-01,  1.99847836e-02, -2.34608958e-03, -3.01979424e-04,
       -8.05562586e-02,  1.53151795e-01, -1.76968835e-02,  1.60615444e-02,
       -5.59441186e-02, -6.52416348e-02,  1.70916989e-01,  3.13425250e-02,
       -4.99401726e-02, -2.30524644e-01, -1.69873863e-01, -6.30855784e-02,
        2.15385899e-01, -1.19875215e-01,  1.00071520e-01,  4.43314873e-02,
        1.39471618e-02, -1.33278817e-01,  3.31550449e-01,  2.31234118e-01,
        1.98473316e-02,  8.94462690e-02, -2.42526889e-01, -2.44034916e-01,
        1.27116684e-02,  2.12468833e-01,  8.57852250e-02,  1.20561369e-01,
        2.16722280e-01, -3.24218392e-01, -7.55669326e-02, -1.11721039e-01,
       -1.05748817e-01,  1.12994939e-01,  4.08040807e-02,  1.08139083e-01,
       -6.39612786e-03,  1.08152069e-01,  1.61990285e-01,  4.39628847e-02,
       -1.33359507e-01,  

In [90]:
at_first_sight_vector

array([-6.64504850e-03,  1.59399619e-03,  1.01922872e-02, -4.77418769e-04,
        8.55779741e-03,  1.99849857e-03,  7.43667316e-03,  1.27941836e-02,
        3.82892089e-03,  1.16451085e-02, -1.00305828e-03,  5.58640715e-03,
        1.42111350e-02,  3.48637556e-03,  2.81161442e-03,  8.20163451e-03,
        1.11545753e-02,  1.52884666e-02, -2.22327770e-03, -2.09946535e-04,
        6.76738238e-03, -2.95032887e-03,  5.02271578e-03, -2.61477032e-03,
        1.30466698e-02, -3.66622233e-04,  1.36561282e-02,  4.97363228e-03,
       -1.69718202e-04,  1.43448757e-02,  1.11149224e-02,  8.18712730e-03,
        2.65743956e-03,  1.04279974e-02,  6.53420459e-04,  2.69812485e-03,
        1.66947953e-03,  8.06423184e-03, -1.37605413e-03,  6.84852432e-03,
        3.99290211e-03,  5.31728147e-03,  6.11834088e-03,  4.20280918e-03,
       -5.73391933e-03,  8.72205012e-03,  1.40553177e-03,  2.76845507e-03,
        8.03082995e-03,  6.59216102e-03,  5.03124436e-03,  1.05207227e-02,
        2.95485486e-03,  

In [91]:
# mymine (나의 영화)와 at_first_sight를 내적하는 코드
np.dot(mymine_vector, at_first_sight_vector)

0.4245981

##### 다른 영화와의 선호도 체크

In [104]:
waiting_to_exhale = movie_to_idx[267]
waiting_to_exhale_vector = als_model.item_factors[waiting_to_exhale]
np.dot(mymine_vector, waiting_to_exhale_vector)

0.01447559

## 6) 모델 활용

##### 좋아하는 영화와 비슷한 영화를 추천받아 보자

In [105]:
favorite_movie = 3225
movieid = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movieid, N=15)
similar_movie

[(2850, 1.0),
 (2605, 0.62121457),
 (3534, 0.58684105),
 (3201, 0.58593863),
 (3323, 0.581492),
 (2694, 0.5749208),
 (2604, 0.574896),
 (3246, 0.5742601),
 (3587, 0.5739141),
 (3324, 0.5720017),
 (3548, 0.57156795),
 (3391, 0.5699417),
 (3329, 0.56840795),
 (3407, 0.5676251),
 (3477, 0.56751007)]

In [107]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[3225,
 3743,
 3779,
 2244,
 2172,
 3149,
 3454,
 744,
 98,
 120,
 3472,
 1364,
 2823,
 3322,
 2592]

In [108]:
def get_similar_movie(movie_name: str):
    movieid = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movieid)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [127]:
def get_similar_title(movie_name: str):
    movieid = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movieid)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    
    title_list = []
    for t in similar_movie:
        title = movies[movies['movie_id'] == t]['title']
        title_list.append(title)
    
    return title_list

In [128]:
get_similar_title(265)

[262    like water for chocolate (como agua para choco...
 Name: title, dtype: object,
 2129    modulations (1998)
 Name: title, dtype: object,
 3015    home page (1999)
 Name: title, dtype: object,
 104    nobody loves me (keiner liebt mich) (1994)
 Name: title, dtype: object,
 2375    24 7: twenty four seven (1997)
 Name: title, dtype: object,
 2891    beefcake (1999)
 Name: title, dtype: object,
 2500    among giants (1998)
 Name: title, dtype: object,
 1056    macao (1952)
 Name: title, dtype: object,
 3143    born to win (1971)
 Name: title, dtype: object,
 597    wooden man's bride, the (wu kui) (1994)
 Name: title, dtype: object]

In [110]:
get_similar_artist(265)

[265, 2198, 3084, 106, 2444, 2960, 2569, 1070, 3212, 601]

In [111]:
get_similar_artist(956)

[956, 3656, 1070, 687, 3517, 3603, 1538, 1555, 1842, 1510]

##### 가장 좋아할 만한 영화들을 추천받아 보자

In [130]:
user = user_to_idx[3041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1450, 0.28673673),
 (2112, 0.26264033),
 (481, 0.25966895),
 (792, 0.25403032),
 (2381, 0.2394364),
 (1499, 0.23858199),
 (1232, 0.23141205),
 (1318, 0.23000002),
 (1786, 0.22144505),
 (829, 0.21832585),
 (1829, 0.21684149),
 (2410, 0.21481536),
 (403, 0.2037984),
 (1476, 0.20010921),
 (1635, 0.19846016),
 (1498, 0.19601408),
 (2631, 0.19277312),
 (1316, 0.19236106),
 (737, 0.19219315),
 (1214, 0.190284)]

In [137]:
reco = [idx_to_movie[i[0]] for i in movie_recommended]
reco

title_list = []
for t in reco:
    title = movies[movies['movie_id'] == t]['title']
    title_list.append(title)
    
title_list

[3497    big kahuna, the (2000)
 Name: title, dtype: object,
 3108    next friday (1999)
 Name: title, dtype: object,
 1421    kolya (1996)
 Name: title, dtype: object,
 3729    what lies beneath (2000)
 Name: title, dtype: object,
 2525    open your eyes (abre los ojos) (1997)
 Name: title, dtype: object,
 3232    whole nine yards, the (2000)
 Name: title, dtype: object,
 3515    breathless (1983)
 Name: title, dtype: object,
 3879    requiem for a dream (2000)
 Name: title, dtype: object,
 3678    jesus' son (1999)
 Name: title, dtype: object,
 2364    civil action, a (1998)
 Name: title, dtype: object,
 2822    happy, texas (1999)
 Name: title, dtype: object,
 3529    hamlet (2000)
 Name: title, dtype: object,
 1847    buffalo 66 (1998)
 Name: title, dtype: object,
 3443    return to me (2000)
 Name: title, dtype: object,
 3546    dinosaur (2000)
 Name: title, dtype: object,
 1451    smilla's sense of snow (1997)
 Name: title, dtype: object,
 2555    after life (1998)
 Name: title, 

In [132]:
what = movie_to_idx[3566]
explain = als_model.explain(user, csr_data, itemid=what)

In [133]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(3538, 0.050232977013887195),
 (3298, 0.04727906288721813),
 (1735, 0.036729132697437855),
 (2769, 0.03216005943750341),
 (3513, 0.03098740422380484),
 (3535, 0.030976673054698055),
 (1258, 0.027607723669111288),
 (3534, 0.02705373394336351),
 (306, 0.02651259954719978),
 (2340, 0.026208190108208185)]

## Result of Project  

현재 다양한 도메인과 현업에서 공통적으로 활용되고 있는 **추천시스템 알고리즘** 관련한 개념과 내용을 학습하였다.   
역시나, 이미지 / 음성 등과 마찬가지로 텍스트가 아닌 **정수** 표현으로 데이터 전처리 및 정제를 해야 했고,   
아무래도 사용자들의 데이터를 기반으로 이루어지는 것이다 보니, 명시적 / 암묵적 평가 기준이 존재했다.    

이를 숫자로 표현한 후에는, Matrix로 표현을 하게 되고, 이를 내적하고 연산하여 유사도를 측정하고, 추천이 이루어지게 되는 알고리즘인 것 같다.   

추천시스템은 아마도 내가 실생활에서 많이 사용하고 있고, 특정 회사의 **사용자**로써 나의 데이터도 다른 사용자의 추천을 위한 데이터로 활용이 되고 있을 것이다.   

하지만 나의 경우, 시간, 장소, 기분, 분위기 등 다양한 주관적이고 변화가 많은 요소들에 의해 선택하게 되는 것들이 많은데, 이러한 일정하지 않은 데이터가 특정 사용자의 추천을 위한 유효한 데이터로 활용이 될 수 있을지 의문이 든다.    

나도 추천을 받는 입장에서 '도대체 왜 나한테 이걸 추천해 주지?'라는 의문이 들 때가 있으니,   
어느 누군가도 그럴 수도 있겠다.    

다양한 요소들이 엮여있고, 지극히 개인적이고 주관적인 요소의 데이터를 활용한 추천시스템에 대하여 관심이 생기는 workshop이었다.

## Good  

데이터를 정제하여, 새로운 분석 단계를 거쳤다.   
새로운 연산(?) 방법인 Matrix Fatorization / CSR Matrix 를 활용하여 연산이 가능했다.   
아주 유용한 **implicit** 패키지를 활용하여 추천시스템 모델을 간단하게 생성할 수 있었다.

## Difficulties / Challenges

movie_id 와 title을 Dictionary 형태로 만드는 전처리 단계를 거치치 않아,   
후작업으로 변경해주는 과정을 추가했더니, 형태가 이상해지는 부분이 생겼다.   

이를 전처리 과정에서 merge한 DataFrame을 활용하여, Dictionary를 제대로 생성하여, 바로 movie_id와 title을 매칭해주는 단계를 거쳐서 수정해야 겠다.   

과정은 어렵지 않았지만, 추천 시스템이 담고 있는 개념들에 대한 깊은 학습이 더 필요할 것 같다.